##### Copyright 2020 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Perdas do TensorFlow Addons: TripletSemiHardLoss

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/losses_triplet"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/losses_triplet.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/losses_triplet.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/addons/tutorials/losses_triplet.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este notebook demonstra como usar a função TripletSemiHardLoss no TensorFlow Addons.

### Recursos:

- [FaceNet: um embedding unificado para o reconhecimento facial e clustering](https://arxiv.org/pdf/1503.03832.pdf)
- [O blog de Oliver Moindrot descreve muito bem o algoritmo em detalhes](https://omoindrot.github.io/triplet-loss)


## TripletLoss

Conforme apresentado pela primeira vez no artigo do FaceNet, a TripletLoss é uma função de perda que treina uma rede neural para incorporar de maneira estreita características da mesma classe enquanto maximiza a distância entre os embeddings de classes diferentes. Para isso, é escolhida uma âncora, além de uma amostra negativa e outra positiva. ![fig3](https://user-images.githubusercontent.com/18154355/61485418-1cbb1f00-a96f-11e9-8de8-3c46eef5a7dc.png)

**A função de perda é descrita como uma função de distância euclidiana:**

![função](https://user-images.githubusercontent.com/18154355/61484709-7589b800-a96d-11e9-9c3c-e880514af4b7.png)

Onde A é a entrada âncora, P é a entrada de amostra positiva, N é a entrada de amostra negativa e alfa é uma margem usada para especificar quando um triplet fica muito "fácil" e você não deve mais ajustar os pesos a partir dele.

## Aprendizado online SemiHard

Conforme mostrado no artigo, os melhores resultados são de triplets conhecidos como "Semi-Hard". Eles são definidos como triplets em que o negativo está mais longe da âncora que do positivo, mas ainda produz uma perda positiva. Para encontrar esses triplets de maneira eficiente, utilize o aprendizado online e só treine a partir de exemplos Semi-Hard em cada lote.


## Configuração

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import io
import numpy as np

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

## Prepare os dados

In [ ]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)

train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

## Crie o modelo

![fig2](https://user-images.githubusercontent.com/18154355/61485417-1cbb1f00-a96f-11e9-8d6a-94964ce8c4db.png)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

## Treine e avalie

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [ ]:
# Train the network
history = model.fit(
    train_dataset,
    epochs=5)

In [ ]:
# Evaluate the network
results = model.predict(test_dataset)

In [ ]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

## Projetor de embeddings

Os arquivos de metadados e vetor podem ser carregados e visualizados aqui: https://projector.tensorflow.org/

Você pode ver os resultados dos nossos dados de teste incorporados quando visualizados com o UMAP: ![embedding](https://user-images.githubusercontent.com/18154355/61600295-e6470380-abfd-11e9-8a00-2b25e7e6916f.png)
